# Day 16: Packet Decoder

In [1]:
HEXTABLE = {
    "0": "0000", "1": "0001", "2": "0010", "3": "0011",
    "4": "0100", "5": "0101", "6": "0110", "7": "0111",
    "8": "1000", "9": "1001", "A": "1010", "B": "1011",
    "C": "1100", "D": "1101", "E": "1110", "F": "1111",
}

def input_sample(sample):
    for char in sample:
        hexadigit = HEXTABLE[char]
        yield hexadigit[0]
        yield hexadigit[1]
        yield hexadigit[2]
        yield hexadigit[3]

    
def input_file(filename):
    with open(filename) as fr:
        yield from input_sample(fr.read())

In [2]:
assert ''.join(input_file('16-sample-1.txt')) ==  '110100101111111000101000'
assert ''.join(input_sample('D2FE28')) ==  '110100101111111000101000'

In [3]:
class Loader:
    
    def __init__(self, stream, tron=False):
        self.stream = stream
        self.consumed = 0
        self.tron = tron
        
    def read(self, num_bits=1):
        if self.tron:
            print(f"read({num_bits})", end=' ')
        self.consumed += num_bits
        buff = [next(self.stream) for _ in range(num_bits)]
        as_bin = ''.join(buff)
        as_int = int(as_bin, 2)
        if self.tron:
            print(as_bin, "[ok]")
        return (as_bin, as_int)

In [4]:
stream = input_file('16-sample-1.txt')
loader = Loader(stream)
assert loader.read(3) == ('110', 6)
assert loader.read(3) == ('100', 4)
assert loader.read(5) == ('10111', 23)
assert loader.read(5) == ('11110', 30)
assert loader.read(5) == ('00101', 5)
assert loader.consumed == 21

In [5]:
class Packet:
    TYPE_LITERAL = 4
    
    def __init__(self, version, type_id, value=None):
        self.version = version
        self.type_id = type_id
        self.value = value
        self.bits_read = 0
        
    def __str__(self):
        if self.type_id == self.TYPE_LITERAL:
            return f'LITERAL {self.value}'
        else:
            return f'OPERATOR {self.type_id}'
        

def read_packet(loader, tron=False):
        
    def read_literal(version, loader):
        buff = []
        bits, _ = loader.read(5)
        while bits[0] == '1': 
            buff.append(bits[1:])
            bits, _ = loader.read(5)
        buff.append(bits[1:])
        value = int(''.join(buff), 2)
        return Packet(version, Packet.TYPE_LITERAL, value)


    def read_operator(version, type_id, loader):
        assert type_id != Packet.TYPE_LITERAL
        length_type_id , _ = loader.read(1)
        operands = []
        if length_type_id == '0':
            sbin, to_consume = loader.read(15)
            if tron:
                print(f' - Total number of bits to consume: {to_consume} 15 bits: {sbin}')
            while to_consume > 0:
                # new_loader = Loader(loader.stream)
                subpacket = read_packet(loader)
                operands.append(subpacket)
                to_consume -= subpacket.bits_read
                if tron:
                    print(f' - Consumed: {subpacket.bits_read} : To consume now is: {to_consume}')

        elif length_type_id == '1':
            _, number_of_packets = loader.read(11)
            if tron:
                print(f' - Total number of packets to consume: {number_of_packets}')
            operands = []
            for i in range(number_of_packets):
                if tron: 
                    print(f"- Readig part {i}", end=" ")
                subpacket = read_packet(loader)
                operands.append(subpacket)
                if tron:
                    print("[OK]")
        else:
            raise ValueError(f"Expected 0 or 1, found {length_type_id!r}")

        return Packet(version, type_id, operands)

    assert isinstance(loader, Loader)
    _start = loader.consumed
    _, version = loader.read(3)
    _, type_id = loader.read(3)
    if type_id == Packet.TYPE_LITERAL:
        result = read_literal(version, loader)
    else:
        result = read_operator(version, type_id, loader)
    result.bits_read = loader.consumed - _start
    return result

Sample 1 is `D2FE28` (In the [16-sample-1.txt](16-sample-1.txt)) file that expands to:

```
110100101111111000101000
VVVTTTAAAAABBBBBCCCCCGGG
```

Below each bit is a label indicating its purpose:

- The three bits labeled V (110) are the packet version, 6.
- The three bits labeled T (100) are the packet type ID, 4, which means the packet is a literal value.
- The five bits labeled A (10111) start with a 1 (not the last group, keep reading) and contain the first four bits of the number, 0111.
- The five bits labeled B (11110) start with a 1 (not the last group, keep reading) and contain four more bits of the number, 1110.
- The five bits labeled C (00101) start with a 0 (last group, end of packet) and contain the last four bits of the number, 0101.
- The three unlabeled 0 bits at the end are extra due to the hexadecimal representation and should be ignored (Gaps in my code).

In [6]:
stream = input_file('16-sample-1.txt')
loader = Loader(stream)
packet = read_packet(loader)
assert packet.version == 6
assert packet.type_id == 4
assert packet.value == 2021

For example, here is an operator packet (hexadecimal string `38006F45291200`) with length type ID $0$ that contains two sub-packets:
(In the [16-sample-2.txt](16-sample-2.txt) file):

```
00111000000000000110111101000101001010010001001000000000
VVVTTTILLLLLLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBBBBBB
```

- The three bits labeled V (`001`) are the packet version, 1.

- The three bits labeled T (`110`) are the packet type ID, 6, which means the packet is an operator.

- The bit labeled I (`0`) is the length type ID, which indicates that the length is a 15-bit number representing the number of bits in the sub-packets.

- The 15 bits labeled L (`000000000011011`) contain the length of the sub-packets in bits, $27$.

- The 11 bits labeled A contain the first sub-packet, a literal value representing the number $10$.

- The 16 bits labeled B contain the second sub-packet, a literal value representing the number $20$.

In [7]:
stream = input_file('16-sample-2.txt')
loader = Loader(stream, tron=True)
p = read_packet(loader)

assert p.version == 1
assert p.type_id == 6
assert len(p.value) == 2
assert p.value[0].value == 10
assert p.value[1].value == 20

read(3) 001 [ok]
read(3) 110 [ok]
read(1) 0 [ok]
read(15) 000000000011011 [ok]
read(3) 110 [ok]
read(3) 100 [ok]
read(5) 01010 [ok]
read(3) 010 [ok]
read(3) 100 [ok]
read(5) 10001 [ok]
read(5) 00100 [ok]


As another example, here is an operator packet (hexadecimal string `EE00D40C823060`, in file [16-sample-3,txt](16-sample-4.txt)) with length type ID `1` that contains three sub-packets:

```
11101110000000001101010000001100100000100011000001100000
VVVTTTILLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBCCCCCCCCCCC
```

- The three bits labeled V (`111`) are the packet version, 7.

- The three bits labeled T (`011`) are the packet type ID, 3, which means the packet is an operator.

- The bit labeled I (`1`) is the length type ID, which indicates that the length is a 11-bit number representing the number of sub-packets.

- The 11 bits labeled L (`00000000011`) contain the number of sub-packets, 3.

- The 11 bits labeled A contain the first sub-packet, a literal value representing the number 1.

- The 11 bits labeled B contain the second sub-packet, a literal value representing the number 2.

- The 11 bits labeled C contain the third sub-packet, a literal value representing the number 3.

In [8]:
stream = input_file('16-sample-3.txt')
loader = Loader(stream)
p = read_packet(loader)

assert p.version == 7
assert p.type_id == 3
assert len(p.value) == 3
assert p.value[0].value == 1
assert p.value[1].value == 2
assert p.value[2].value == 3

## Solution part one

In [9]:
def find_solution_one(sequence, tron=False):
    loader = Loader(sequence)
    packet = read_packet(loader)
    
    def sum_version(p, level=0):
        if tron:
            print(" " * level, p, 'Version:', p.version)
        if p.type_id == Packet.TYPE_LITERAL:
            return p.version
        else:
            return p.version + sum(
                sum_version(subp, level=level+1) for subp in p.value
            )
    return sum_version(packet)

In [10]:
assert find_solution_one(input_file('16-sample-1.txt')) == 6
assert find_solution_one(input_file('16-sample-2.txt')) == 9

Here are a few more examples of hexadecimal-encoded transmissions:

**`8A004A801A8002F478`** represents an operator packet (version $4$) which contains an operator packet (version $1$) which contains an operator packet (version $5$) which contains a literal value (version $6$); this packet has a version sum of $16$.

In [11]:
assert find_solution_one(input_sample('8A004A801A8002F478')) == 16

**`620080001611562C8802118E34`** represents an operator packet (version $3$) which contains two sub-packets; each sub-packet is an operator packet that contains two literal values. This packet has a version sum of $12$.

In [12]:
assert find_solution_one(input_sample('620080001611562C8802118E34')) == 12

**`C0015000016115A2E0802F182340`** has the same structure as the previous example, but the outermost packet uses a different length type ID. This packet has a version sum of $23$.

In [13]:
assert find_solution_one(input_sample('C0015000016115A2E0802F182340')) == 23     

**`A0016C880162017C3686B18A3D4780`** is an operator packet that contains an operator packet that contains an operator packet that contains five literal values; it has a version sum of 31.

In [14]:
sol = find_solution_one(input_file('16-input.txt'))
print(f"Solution for part one: {sol}")

Solution for part one: 971


## Second part

In [15]:
from functools import reduce

In [16]:

TYPE_SUM = 0
TYPE_PRODUCT = 1
TYPE_MINIMUN = 2
TYPE_MAXIMUM = 3
TYPE_LITERAL = 4
TYPE_GREATER = 5
TYPE_LESS = 6
TYPE_EQUAL = 7

LITERAL_TYPES = ['SUM', 'PRODUCT', 'MIN', 'MAX', 'LIT', 'GREATER', 'LESS', 'EQUAL']

def evaluate(packet, level=0, tron=True):
    if tron:
        print(" " * level, packet, LITERAL_TYPES[packet.type_id])
    if packet.type_id == TYPE_LITERAL:
        return packet.value
    items = [evaluate(sp, level=level+1, tron=tron) for sp in packet.value]
    if packet.type_id == TYPE_SUM:
        return sum(items)
    elif packet.type_id == TYPE_PRODUCT:
        return reduce(lambda x, y: x*y, items, 1)
    elif packet.type_id == TYPE_MINIMUN:
        return min(items)
    elif packet.type_id == TYPE_MAXIMUM:
        return max(items)
    elif packet.type_id == TYPE_GREATER:
        return items[0] > items[1]
    elif packet.type_id == TYPE_LESS:
        return items[0] < items[1]
    elif packet.type_id == TYPE_EQUAL:
        return items[0] == items[1]
    raise ValueError('Packet type expected in range [1..7] but {packet.type_pf|} found!')

In [17]:
stream = input_file('16-sample-1.txt')
loader = Loader(stream)
packet = read_packet(loader)
assert evaluate(packet) == 2021

 LITERAL 2021 LIT


In [18]:
def find_solution_two(sequence, tron=True):
    loader = Loader(sequence)
    packet = read_packet(loader)
    return evaluate(packet, tron=tron)

**`C200B40A82`** finds the sum of 1 and 2, resulting in the value $3$.

In [19]:
stream = input_sample('C200B40A82')
assert find_solution_two(stream, tron=True) == 3

 OPERATOR 0 SUM
  LITERAL 1 LIT
  LITERAL 2 LIT


**`04005AC33890`** finds the product of 6 and 9, resulting in the value $54$.


In [20]:
stream = input_sample('04005AC33890')
assert find_solution_two(stream) == 54

 OPERATOR 1 PRODUCT
  LITERAL 6 LIT
  LITERAL 9 LIT


**`880086C3E88112`** finds the minimum of $7$, $8$, and $9$, resulting in the value $7$.

In [21]:
stream = input_sample('880086C3E88112')
assert find_solution_two(stream) == 7

 OPERATOR 2 MIN
  LITERAL 7 LIT
  LITERAL 8 LIT
  LITERAL 9 LIT


**`CE00C43D881120`** finds the maximum of 7, 8, and 9, resulting in the value 9.

In [22]:
stream = input_sample('CE00C43D881120')
assert find_solution_two(stream) == 9

 OPERATOR 3 MAX
  LITERAL 7 LIT
  LITERAL 8 LIT
  LITERAL 9 LIT


**`D8005AC2A8F0`** produces $1$, because $5$ is less than $15$.

In [23]:
stream = input_sample('D8005AC2A8F0')
assert find_solution_two(stream) == 1

 OPERATOR 6 LESS
  LITERAL 5 LIT
  LITERAL 15 LIT


**`F600BC2D8F`** produces $0$, because $5$ is not greater than $15$.

In [24]:
stream = input_sample('F600BC2D8F')
assert find_solution_two(stream) == 0

 OPERATOR 5 GREATER
  LITERAL 5 LIT
  LITERAL 15 LIT


**`9C005AC2F8F0`** produces $0$, because $5$ is not equal to $15$.

In [25]:
stream = input_sample('9C005AC2F8F0')
assert find_solution_two(stream) == 0

 OPERATOR 7 EQUAL
  LITERAL 5 LIT
  LITERAL 15 LIT


**`9C0141080250320F1802104A08` produces $1$, because $1 + 3 = 2 \times 2$.

In [26]:
stream = input_sample('9C0141080250320F1802104A08')
assert find_solution_two(stream) == 1

 OPERATOR 7 EQUAL
  OPERATOR 0 SUM
   LITERAL 1 LIT
   LITERAL 3 LIT
  OPERATOR 1 PRODUCT
   LITERAL 2 LIT
   LITERAL 2 LIT


In [27]:
sol = find_solution_two(input_file('16-input.txt'), tron=False)
print(f"Solution for part one: {sol}")

Solution for part one: 831996589851


## Extra

Nice tree:

In [28]:
sol = find_solution_two(input_file('16-input.txt'), tron=True)
print(f"Solution for part one: {sol}")

 OPERATOR 0 SUM
  OPERATOR 1 PRODUCT
   OPERATOR 6 LESS
    LITERAL 79 LIT
    LITERAL 18 LIT
   LITERAL 130 LIT
  OPERATOR 1 PRODUCT
   OPERATOR 6 LESS
    LITERAL 17 LIT
    LITERAL 3739 LIT
   LITERAL 26181 LIT
  OPERATOR 3 MAX
   LITERAL 3381457610 LIT
   LITERAL 13 LIT
  OPERATOR 0 SUM
   LITERAL 368803 LIT
   LITERAL 46 LIT
   LITERAL 30827667693 LIT
   LITERAL 172659535 LIT
  OPERATOR 1 PRODUCT
   OPERATOR 6 LESS
    LITERAL 30901 LIT
    LITERAL 11 LIT
   LITERAL 229877055 LIT
  OPERATOR 1 PRODUCT
   LITERAL 796 LIT
   OPERATOR 7 EQUAL
    OPERATOR 0 SUM
     LITERAL 9 LIT
     LITERAL 15 LIT
     LITERAL 7 LIT
    OPERATOR 0 SUM
     LITERAL 11 LIT
     LITERAL 4 LIT
     LITERAL 6 LIT
  OPERATOR 1 PRODUCT
   OPERATOR 5 GREATER
    LITERAL 32638 LIT
    LITERAL 32638 LIT
   LITERAL 3874 LIT
  LITERAL 43547 LIT
  LITERAL 9965 LIT
  OPERATOR 2 MIN
   LITERAL 482015 LIT
   LITERAL 3103 LIT
   LITERAL 654457 LIT
   LITERAL 57514 LIT
   LITERAL 6 LIT
  LITERAL 3139 LIT
  OPERATOR 3